In [38]:
from bs4 import BeautifulSoup
import time
from openpyxl import Workbook
import re
from selenium import webdriver
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By

In [ ]:


service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

url = "https://www.lnr.fr/"

driver.get(url)

top_14_href = driver.find_elements(By.XPATH,'//div[@class="small-6 columns text-center"][1]/a[1]')[0]
pro_d2_href = driver.find_elements(By.XPATH,'//div[@class="small-6 columns text-center"][2]/a[1]')[0]
cookie_btn = driver.find_elements(By.XPATH,'//span[@class="didomi-continue-without-agreeing"]')

top_14_link = top_14_href.get_attribute("href")
pro_d2_link = pro_d2_href.get_attribute("href")

# Don't accept cookie
cookie_btn[0].click()

# TOP 14 page
top_14_href.click() # Go to top 14 page
players_href = driver.find_elements(By.XPATH,'//nav[@class="top-bar subnav sticky-nav"]/section/ul[@class="right"]/li[7]/a')[0]
# Open players page
players_href.click()

# Select filter by team
alphabetical_filter_href = driver.find_elements(By.XPATH,'//dl/dd[2]/a')[0]
alphabetical_filter_href.click()

# Link from A to Z
letters_filter_href = driver.find_elements(By.XPATH,'//div[@class="view"][1]//ul[@class="type-letters"]/li')
print(len(letters_filter_href))

links = []
for curr_letter_link in letters_filter_href:
    curr_letter_link.click()

    # Wait 2 seconds between each letters
    time.sleep(2)

    players_card_href_list = driver.find_elements(By.XPATH,'//ul[@class="fluid-block-grid-7 player-list vs-items"]//li/a') 
    
    for player_card in players_card_href_list:
        links.append(player_card.get_attribute("href"))

# time.sleep(360)
driver.close()

In [61]:
len(links)

584

In [63]:
book = Workbook()
sheet = book.active
error_sheet = book.create_sheet("errors")


sheet.cell(row=1, column=1).value = "Nom et prenom"
sheet.cell(row=1, column=2).value = "Date de naissance"
sheet.cell(row=1, column=3).value = "Taille (en cm)"
sheet.cell(row=1, column=4).value = "Poid (en kg)"
sheet.cell(row=1, column=5).value = "Poste"
sheet.cell(row=1, column=6).value = "Club actuel"

error_sheet.cell(row=1, column=1).value = "Page URL"
error_sheet.cell(row=1, column=2).value = "Error"

i = 2
j = 2

service = ChromeService(executable_path=ChromeDriverManager().install())

for link in links:
    print(link)
    
    options = webdriver.ChromeOptions() 
    options.add_argument('--headless')
    errors = []
    
    with webdriver.Chrome(service=service, options=options) as driver:  
        driver.get(link)

        try:
            name = driver.find_element(By.XPATH,'//h1[@id="page-title"]')
            sheet.cell(row=i, column=1).value = name.text
        except Exception as e:
            sheet.cell(row=i, column=1).value = ""

            error_sheet.cell(row=j, column=1).value = link
            error_sheet.cell(row=j, column=2).value = e.__str__()
            j += 1
        try:
            # With regex
            birthdate = driver.find_element(By.XPATH,'//span[contains(., "Date")]/following-sibling::span')
            sheet.cell(row=i, column=2).value = birthdate.text
        except Exception as e:
            sheet.cell(row=i, column=2).value = ""

            error_sheet.cell(row=j, column=1).value = link
            error_sheet.cell(row=j, column=2).value = e.__str__()
            j += 1
        try:
            # With regex
            height = driver.find_element(By.XPATH,'//span[contains(., "Taille")]/following-sibling::span')
            sheet.cell(row=i, column=3).value = int(height.text.split(" ")[0])
        except Exception as e:
            sheet.cell(row=i, column=3).value = ""

            error_sheet.cell(row=j, column=1).value = link
            error_sheet.cell(row=j, column=2).value = e.__str__()
            j += 1
        try:
            # With regex
            weight = driver.find_element(By.XPATH,'//span[contains(., "Poids")]/following-sibling::span')
            sheet.cell(row=i, column=4).value = int(weight.text.split(" ")[0])
        except Exception as e:
            sheet.cell(row=i, column=4).value = ""

            error_sheet.cell(row=j, column=1).value = link
            error_sheet.cell(row=j, column=2).value = e.__str__()
            j += 1
        try:
            position = driver.find_element(By.XPATH,'//div[@class="visu-infos"]/h2')
            sheet.cell(row=i, column=5).value = position.text
        except Exception as e:
            sheet.cell(row=i, column=5).value = ""

            error_sheet.cell(row=j, column=1).value = link
            error_sheet.cell(row=j, column=2).value = e.__str__()
            j += 1
        try:
            team = driver.find_element(By.XPATH,'//ul[@class="infos-list"]/li/h3')
            sheet.cell(row=i, column=6).value = team.text
        except Exception as e:
            sheet.cell(row=i, column=6).value = ""

        i += 1
        time.sleep(2)

book.save("liste_joueurs_top_14.xlsx")

https://www.lnr.fr/joueurs/sylvain-abadie
https://www.lnr.fr/rugby-top-14/joueurs/paul-abadie
https://www.lnr.fr/rugby-top-14/joueurs/esteban-abadie
https://www.lnr.fr/joueurs/abdelkrim-abbad
https://www.lnr.fr/rugby-pro-d2/joueurs/nicholas-abendanon
https://www.lnr.fr/rugby-pro-d2/joueurs/antoine-abraham
https://www.lnr.fr/rugby-top-14/joueurs/sergy-abramishvili
https://www.lnr.fr/rugby-top-14/joueurs/mathieu-acebes
https://www.lnr.fr/joueurs/juan-jose-acosta
https://www.lnr.fr/rugby-top-14/joueurs/thomas-acquier
https://www.lnr.fr/joueurs/nicholas-adams
https://www.lnr.fr/joueurs/heini-adams
https://www.lnr.fr/joueurs/bastien-adrillon
https://www.lnr.fr/joueurs/viliamu-afatia
https://www.lnr.fr/rugby-top-14/joueurs/zaccharie-affane
https://www.lnr.fr/rugby-top-14/joueurs/louis-antonin-agostini
https://www.lnr.fr/rugby-top-14/joueurs/pita-jordan-ahki
https://www.lnr.fr/rugby-top-14/joueurs/stephane-ahmed
https://www.lnr.fr/joueurs/lusiano-aisake
https://www.lnr.fr/rugby-top-14/joueurs

In [53]:
len(links)


584